In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


#!/usr/bin/env python


# In[1]:


#!/usr/bin/env python


# In[1]:


'''Example of VAE on MNIST dataset using MLP

The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean = 0 and std = 1.

# Reference

[1] Kingma, Diederik P., and Max Welling.
"Auto-Encoding Variational Bayes."
https://arxiv.org/abs/1312.6114
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from csv import reader

# import

from keras.layers import Lambda, Input, Dense
from keras.layers import Dropout
# keras lambda layer
from keras.models import Model
# from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

import tensorflow as tf
# from keras.layers import Input, Dense
# from keras.models import Model
import csv
from os import listdir
from os.path import isfile, join
from scipy.misc import imread
# import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random as rd
import cv2

# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon


# In[2]:

# inputno = 0/0
# inp11 = 0
# out11 = 0

# def getnewloss(x2):

#     outputno = outputno + 1
#     return binary_crossentropy(x1,x2)


# def getloss(x1):
#     # output = outputs[inputno]
#     # inputindex = imindex[inputno]
#     outputno = 0
#     losses1 = tf.map_fn(lambda x : getnewloss(x),outputs)
#     inputno = inputno + 1
#     return tf.reduce_sum(losses1)

def classcrossentropy(inputs,outputs):
    loss = 0
    i = 0

    inputs = inputs[1]
    # outputs = 
    recon_loss = -tf.reduce_sum(
    inputs * tf.log(1e-10+outputs) + 
    (1-inputs) * tf.log(1e-10+1-outputs), 
    axis=1
)

    return recon_loss



    # print(tf.shape(inputs))
    # print(outputs.shape)
    # print(inputs[0].shape)
    # print(inputs[1].shape)
    
    # print(inputs.shape)
    #
    # return binary_crossentropy(inputs[1],outputs)
    
#     tf.map_fn(
#     fn,
#     elems,
#     dtype=None,
#     parallel_iterations=None,
#     back_prop=True,
#     swap_memory=False,
#     infer_shape=True,
#     name=None
# )
                                   
    # inp11 = inputs[0]
    # out11 = outputs                                                     
    # iminput = inputs[0]
    # imindex = inputs[1]
    # indices = [i for i, x in enumerate(my_list) if x == "whatever"]
    # outputs[:] = outputs[:].append(imindex)
    # for rec in outputs:
        # loss = rec
    # outputno = 0
    # inputno = 0
    # losses = tf.map_fn(lambda x : getloss(x),)

    # return tf.reduce_sum(losses,axis = 1)

    # inputs = inputs[0]

    # epsilon = 1e-10
    # epsilon = epsilon*np.ones((128,7500))

    # recon_loss = -tf.reduce_sum(
    #     inputs * tf.log(1e-10+outputs) + 
    #     (1-inputs) * tf.log(1e-10+1-outputs), 
    #     axis=1
    # )

    # recon_loss = tf.reduce_mean(recon_loss)

    # Latent loss
    # KL divergence: measure the difference between two distributions
    # Here we measure the divergence between 
    # the latent distribution and N(0, 1)
    # latent_loss = -0.5 * tf.reduce_sum(
    #     1 + z_log_sigma_sq - tf.square(z_mu) - 
    #     tf.exp(z_log_sigma_sq), axis=1)
    # latent_loss = tf.reduce_mean(latent_loss)

    # total_loss = recon_loss + latent_loss
    # return recon_loss

    # j = 0
    # # return binary_crossentropy(inputs[0],outputs)
    # inputim = inputs[0]
    # inputlabel = inputs[1]

    # for i in range(batch_size):
    #     # j = 0
    #     for j in range(batch_size):
    #         # j+= 1
    #         # input1 = inputim[i]
    #         input2 = inputim[j]
    #         k1 = inputlabel[i]
    #         k2 = inputlabel[j]
    #         output1 = outputs[i]
    #         # print(k1)
    #         # print(k2)
    #         # if(k1 == k2):
    #         loss = loss + binary_crossentropy(input2,output1)
    #     # i+= 1
    # return loss




    # print(inputs1.shape)
    # # if(runmode == "single"):
    # # return binary_crossentropy(inputs,outputs)
    

    # i = 0
    # for inputs in inputs1:
    #     outputs = outputs1[i]
    #     imlabel = 0
    #     i1 = 0
    #     for im1 in flowerlist:
    #         # if(im1 == inputs):
    #         if(np.array_equal(im1,inputs)):
    #             imlabel = labels[i1]
    #             break
    #         i1 = i1+1
    #     print(imlabel)
    # #     for key in dict1:    
    #     loss = 0
    #     i1 = 0
    #     for image1 in flowerlist:
    #         if(labels[i1] == imlabel):
    #             if(loss == 0):
    #                 loss = binary_crossentropy(image1,outputs)
    #             else:
    #                 loss = loss + binary_crossentropy(image1,outputs)
    #         i1 += 1
    #     # return binary_crossentropy(inputs,outputs)
    #     print(loss)
    #     print(binary_crossentropy(image1,outputs))
    #     i = i+1
    # return K.sum(loss)


# In[2]:


def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


# In[3]:


# In[3]:


def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as a function of the 2D latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = (n - 1) * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()


# In[6]:


# In[4]:
# folder = "jpg"
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


# In[9]:


# In[17]:


dict1 = {}
mypath = "/home/anurag/Desktop/projects/VAEproj/102flowers/jpg"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# sort(onlyfiles)
onlyfiles.sort()
filename = 'FileName.csv'

labels = load_csv(filename)
labels = labels[0]    
print(onlyfiles)
# i1 = 1
# seltol = 1300
#seltol = 10
# selected1 = 800
# for selected1 in [200]:
# samplpositive = 600
#samplpositive = 10
featurestotal = []
i1 = 0
for image in onlyfiles:
    if(image[-3:] != "jpg"):
        continue
    # i1 = i1 + 1
    # print("efrffrwf")

    im1 = mypath+"/"+ image
    # print(im1)
    # print(labels[i1])
    # im = imread(im1,cv2.COLOR_BGR2GRAY)
    im = imread(im1)
    im = im/float(255)
    # print(im)
    res = cv2.resize(im, dsize=(50,50), interpolation=cv2.INTER_CUBIC)
    #res = cv2.resize(im, dsize=(500,500))
    # res = im
    # k1


    k1 = np.reshape(res,-1)
    # np.append(k1,labels[i1])
    # print(k)
    k1 = list(k1)
    k1.append(labels[i1])
    # print(k1)

#     dict1[k1] = labels[i1] 
    # print(k1.shape)
        # print(i1)
    print(i1)
    featurestotal.append(k1)
    i1 = i1+ 1
    # if(i1 == 85):
        # break



flowerlist = []
rd.shuffle(featurestotal)
# flowerlist.extend(featurestotal)

# flowerlist contains the required entries

Using TensorFlow backend.


['image_00001.jpg', 'image_00002.jpg', 'image_00003.jpg', 'image_00004.jpg', 'image_00005.jpg', 'image_00006.jpg', 'image_00007.jpg', 'image_00008.jpg', 'image_00009.jpg', 'image_00010.jpg', 'image_00011.jpg', 'image_00012.jpg', 'image_00013.jpg', 'image_00014.jpg', 'image_00015.jpg', 'image_00016.jpg', 'image_00017.jpg', 'image_00018.jpg', 'image_00019.jpg', 'image_00020.jpg', 'image_00021.jpg', 'image_00022.jpg', 'image_00023.jpg', 'image_00024.jpg', 'image_00025.jpg', 'image_00026.jpg', 'image_00027.jpg', 'image_00028.jpg', 'image_00029.jpg', 'image_00030.jpg', 'image_00031.jpg', 'image_00032.jpg', 'image_00033.jpg', 'image_00034.jpg', 'image_00035.jpg', 'image_00036.jpg', 'image_00037.jpg', 'image_00038.jpg', 'image_00039.jpg', 'image_00040.jpg', 'image_00041.jpg', 'image_00042.jpg', 'image_00043.jpg', 'image_00044.jpg', 'image_00045.jpg', 'image_00046.jpg', 'image_00047.jpg', 'image_00048.jpg', 'image_00049.jpg', 'image_00050.jpg', 'image_00051.jpg', 'image_00052.jpg', 'image_0005

/home/anurag/.local/lib/python2.7/site-packages/ipykernel_launcher.py:369: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
28

1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071


3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719


5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372


6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022


In [2]:
x_train = featurestotal[:7000]
x_test = featurestotal[7000:]


# x_train = featurestotal[:4000]
# x_test = featurestotal[4000:]


# x_train = featurestotal[:400]
# x_test = featurestotal[400:]


x_train = np.array(x_train,dtype = float)
x_test = np.array(x_test,dtype = float)

print(x_train)
print(x_test)


x_trainlabel = x_train[:,7500]
x_train = x_train[:,0:7500]
x_testlabel = x_test[:,7500]
x_test = x_test[:,0:7500]


a = np.zeros(7500,dtype = float)
 # = np.repeat(a[:, :, np.newaxis], 3, axis=2)

x_trainlabelsum = np.repeat(a[np.newaxis,:], 102,axis = 0)
x_testlabelsum = np.repeat(a[np.newaxis,:], 102,axis = 0)
print(x_trainlabelsum[0].shape)

# x_testlabelsum = []


# In[18]:


trainnum = np.zeros(102)
testnum = np.zeros(102)

# In[18]:


i = 0
for fq in x_trainlabel:
    image = x_train[i]
    fq = int(fq)
    fq = fq-1
#     print(fq)
#     print(x_trainlabelsum[fq].shape)
#     print(type(image))
#     print(image.shape)
#     print(x_trainlabelsum[fq])
# #     print(type(image))
#     print(image)
    
    x_trainlabelsum[fq] = np.add(x_trainlabelsum[fq],image)
    trainnum[fq] = trainnum[fq] + 1
    i = i+1

i = 0
for fq in x_testlabel:
    fq = fq-1
    im = x_test[i]
    fq = int(fq)
    x_testlabelsum[fq] = np.add(x_testlabelsum[fq],im)
    testnum[fq] = testnum[fq] + 1
    i = i+1


x_train2 = np.copy(x_train)
x_test2 = np.copy(x_test)

i = 0
for d2 in x_trainlabel:
    d2 = int(d2)
    d2 = d2-1
    x_train2[i] = x_trainlabelsum[d2]/(float(trainnum[d2]))
    i = i+1

i = 0
for d2 in x_testlabel:
    d2 = int(d2)
    d2 = d2-1
    x_test2[i] = x_testlabelsum[d2]/(float(trainnum[d2]))
    i = i+1

# x_train

# x_trainlabel = np.array(x_trainlabel)
# x_testlabel = np.array(x_testlabel)

# x_train


# print(classcrossentropy(x_train[0],x_train[1]))
# MNIST dataset
# (x_train, y_train), (x_test, y_test) = mnist.load_data()


# In[3]:


# In[19]:

[[3.78881368e-01 2.69599935e-01 2.13714092e-01 ... 3.58309599e-01
  2.57681160e-01 8.70000000e+01]
 [3.96971060e-01 4.30781767e-01 3.10597818e-01 ... 3.57862406e-01
  4.89520443e-02 3.50000000e+01]
 [9.71846274e-01 9.77632754e-01 9.57597993e-01 ... 7.47822265e-01
  8.51600570e-01 7.20000000e+01]
 ...
 [1.31404611e-01 2.22750262e-01 2.31551246e-02 ... 1.96516180e-01
  1.89426502e-02 5.10000000e+01]
 [2.70588235e-01 4.70955882e-01 2.50245098e-01 ... 3.40776921e-01
  1.27126487e-01 4.60000000e+01]
 [6.08436120e-02 6.86867492e-02 1.37282954e-02 ... 2.02054757e-01
  8.43374704e-02 5.10000000e+01]]
[[ 1.15687202e-01  1.11080074e-01  2.98892451e-02 ...  1.90214934e-01
   6.99943461e-02  5.90000000e+01]
 [ 9.79686658e-02  5.87529795e-02  5.09098422e-02 ...  3.89835009e-01
   2.85518265e-01  5.00000000e+00]
 [ 3.05325533e-01  2.89639258e-01  2.93560827e-01 ...  5.28648440e-01
   4.10633734e-01  6.60000000e+01]
 ...
 [ 1.72794118e-02  1.33578431e-02 -3.67647059e-04 ...  4.01877397e-02
   8.46480

In [5]:
# image_size = x_train[0].shape[0]
# print(image_size)
original_dim = 7500
# x_train = np.reshape(x_train, [-1, original_dim])
# x_test = np.reshape(x_test, [-1, original_dim])
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255
# print(im)
# network parameters
input_shape = (original_dim, )
# print("input_shape")
# print(input_shape)
intermediate_dim = 512
intermediate_dim1 = 512

batch_size = 128
latent_dim = 2
epochs = 50


# model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# VAE model = encoder + decoder
# build encoder model
input1 = Input(shape=input_shape, name='encoder_input_im')
input2 = Input(shape=input_shape, name='encoder_input_imsum')

# inputs = Input(shape=1, name='encoder_input_label')
# drop1 = Dropout(0.2)(input1)
x = Dense(intermediate_dim, activation='relu')(input1)
x1 = Dense(intermediate_dim1, activation='relu')(x)
# drop1 = Dropout(0.2)(x)
z_mean = Dense(latent_dim, name='z_mean')(x1)
z_log_var = Dense(latent_dim, name='z_log_var')(x1)

inps = [input1,input2]
# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model

# encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder = Model(inputs = [input1,input2], [z_mean, z_log_var, z], name='encoder')
encoder = Model(inps, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
# plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')

x = Dense(intermediate_dim, activation='relu')(latent_inputs)
# drop2 = Dropout(0.2)(x)
x1 = Dense(intermediate_dim1, activation='relu')(x)

outputs = Dense(original_dim, activation='sigmoid')(x1)


# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
# plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inps)[2])
vae = Model(inps, outputs, name='vae_mlp')


# inputs = Input(shape=input_shape, name='encoder_input')
# x = Dense(intermediate_dim, activation='relu')(inputs)
# z_mean = Dense(latent_dim, name='z_mean')(x)
# z_log_var = Dense(latent_dim, name='z_log_var')(x)

# # use reparameterization trick to push the sampling out as input
# # note that "output_shape" isn't necessary with the TensorFlow backend
# z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# # instantiate encoder model
# encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()
# # plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# # build decoder model
# latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
# x = Dense(intermediate_dim, activation='relu')(latent_inputs)
# outputs = Dense(original_dim, activation='sigmoid')(x)

# # instantiate decoder model
# decoder = Model(latent_inputs, outputs, name='decoder')
# decoder.summary()
# # plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# # instantiate VAE model
# outputs = decoder(encoder(inputs)[2])
# vae = Model(inputs, outputs, name='vae_mlp')

if __name__ == '__main__':
    # dict1 = {}
    kllosswt = 1;
    parser = argparse.ArgumentParser()

    help_ = "Load h5 model trained weights"
    parser.add_argument("-w", "--weights", help=help_)
    help_ = "Use mse loss instead of binary cross entropy (default)"
    parser.add_argument("-m",
                        "--mse",
                        help=help_, action='store_true')
    args = parser.parse_args()
    models = (encoder, decoder)
    # data = (x_test, y_test)

# here1
    # VAE loss = mse_loss or xent_loss + kl_loss
    
    # runmode = input("single or class encoder ")
    # z33 = np.identity(1000)


    if args.mse:
        reconstruction_loss = mse(inputs, outputs)
    else:
        reconstruction_loss = classcrossentropy(inps,
                                                  outputs)

    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss = kl_loss*-0.5*kllosswt
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    vae.summary()
    # plot_model(vae,
               # to_file='vae_mlp.png',
               # show_shapes=True)

    if args.weights:
        vae.load_weights(args.weights)
    else:
        # train the autoencoder
        vae.fit([x_train,x_train2],
                epochs=epochs,
                batch_size=batch_size,
                validation_data=([x_test,x_test2], None))
        vae.save_weights('vae_mlp_mnist.h5')



    # for i in test2
    # plot_results(models,
                 # data,
                 # batch_size=batch_size,
                 # model_name="vae_mlp")

    encoder, decoder = models
    # encoder, decoder = models
    x_test, y_test = [x_train,x_train2],x_trainlabel
    # os.makedirs(model_name, exist_ok=True)

    # filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    # plt.savefig(filename)
    plt.show()
# In[ ]:


# In[ ]:

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_im (InputLayer)   (None, 7500)         0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 512)          3840512     encoder_input_im[0][0]           
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 512)          262656      dense_8[0][0]                    
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            1026        dense_9[0][0]                    
__________________________________________________________________________________________________
z_log_var 

usage: ipykernel_launcher.py [-h] [-w WEIGHTS] [-m]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-329283ae-48b9-4ab5-938f-7faba10522d5.json


SystemExit: 2

/home/anurag/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
